# 
apogee_id: Unique identifyer to find same object in different fiels Average: 
        N_Visits number of times we've seen it VHelio_avg: Average velocity relative to our sun accross all measurments (km/s) AllVis table: VHelio: Measurments (km/s) uncertianty VRelerr (km/s) AvgTable: VScatter: RMS scatter from each individual measurment.
Compairs the uncertainty to the scatter. 
    StableRV_xi2: xi^2 of the measurments around the mean value. 
    StableRV_xi2_prob: P value of the xi^2 value with the number of DOF. 
    Jd: Julian Date, time that we saw it. 1) VScatter StableRV_chi2_prob Histogram of the data, 2) THen look at properties, color magnitude diagram. (CMD) H vs. J-K (average file Near IR) make binary ones in different color fromt the rest to see patterns, Fe_h alpha_m (Iron/H ratio) alpha_m

h - (j/k)


# 
Author:    Cody Brown
Date:      February 28, 2018
Purpose:   This takes in the simplified data made by Data_Simplify and uses it with The Joker to get paramiter estimation     for possible binaries.

In [1]:
from thejoker.data import RVData
from thejoker.plot import plot_rv_curves
from thejoker.sampler import compute_likelihoods
import astropy.units as u
import schwimmbad
from astropy.io import fits
from astropy.table import Table

import os

import numpy as np
import matplotlib.pyplot as plt


AllAvgSimp = Table(fits.getdata('/home/cody/Binary Search/AllAvgSimp.fits',1))
AllVisSimp   = Table(fits.getdata('/home/cody/Binary Search/AllVisSimp.fits',1))

/home/cody/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [ ]:
# Now I have my table of Visits I want to look at. But now I'd like a bunch of little arrays to make things 
# a little easier to plot and I'd like to only keep 'JD' and 'VHELIO' I can probably use the 'NVISITS' in teh AllAverage
# to help organize things.
#First step is make empty arrays using AllAvgSimp with length 'NVISITS


start = 0
end = 0
loop = 0
name = 0

for n in AllAvgSimp['NVISITS']:
    start = end
    end = end + n
    Date = AllVisSimp['JD'][start:end] - AllVisSimp['JD'][start]
    RV = AllVisSimp['VHELIO'][start:end]* u.km/u.s
    err = AllVisSimp['VRELERR'][start:end] * u.km/u.s
    data = RVData( t = Date, rv = RV, stddev = err)
    ax = data.plot()
    ax.set_xlabel('Time [day]')
    ax.set_ylabel('RV [km/s]')
    thing = AllAvgSimp['APOGEE_ID'][name]
    plt.title(thing)
    
    from thejoker.sampler import JokerParams
    return_logprobs = True
    params = JokerParams(P_min = 8*u.day, P_max= 512*u.day)
    
    from thejoker.sampler import TheJoker
    joker = TheJoker(params)
    samples = joker.rejection_sample(data,n_prior_samples = 2**16,)
    
    #COuld make a plot of parmiters but I don't really want to 
    #fig, ax = plt.subplots(1,1)
    #ax.scatter(samples['P'].value, samples['K'].to(u.km/u.s).value, 
    #          marker ='.', color = 'k', alpha = 0.45)
    #ax.set_xlabel('$P$ [day]')
    #ax.set_ylabel('$K$ [km/s]')
    
    #ax.scatter(61.942, 1.3959, marker='o', color = '#31a354', zorder=-100)   
    
    fig, ax = plt.subplots(1,1)
    t_grift = np.linspace(-10,80,1024)
    plot_rv_curves(samples, t_grift, rv_unit=u.km/u.s, data = data,
                  ax = ax, plot_kwargs=dict(color='#888888'))
    #plt.show(ax)
    newpath = r'/home/cody/Binary Search/'+str(AllAvgSimp['APOGEE_ID'][name])
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    #Saving the plot with all the samples
    #name_holder = AllVisSimp['APOGEE_ID'][name]
    fig.savefig('/home/cody/Binary Search/'+str(AllAvgSimp['APOGEE_ID'][name]) +'/P' +str(AllAvgSimp['APOGEE_ID'][name]) + '.png')
    
    plt.close()
    #Save the paramiters that The Joker got for the orbit. The varable 'samples' is some weird type of object that I can't
    #figure out how\ to save. So instead lets turn it into a table, save that, and then it'll be easier to work wtih?
    Samples_Table = Table()
    Samples_Table['P'] = samples['P']
    Samples_Table['M0'] = samples['M0']
    Samples_Table['e'] = samples['e']
    Samples_Table['omega'] = samples['omega']
    Samples_Table['jitter'] = samples['jitter']
    Samples_Table['K'] = samples['K']
    Samples_Table['v0'] = samples['v0']
    Samples_Table.write("/home/cody/Binary Search/"+str(AllAvgSimp['APOGEE_ID'][name])+"/" +str(AllAvgSimp['APOGEE_ID'][name])+'.fits', format = 'fits' )
    #loop += 1
    name += 1
    #print(name/len(AllVisSimp['APOGEE_ID']))
    #if loop > 10:
    #    break
    #if name == int(len(AllAvgSimp['APOGEE_ID'])*.25):
    #    print(" 25% Done")
    #if name == int(len(AllAvgSimp['APOGEE_ID'])*0.5):
    #    print(" 50% Done")
    #if name == int(len(AllAvgSimp['APOGEE_ID'])*.75):
    #    print(" 75% Done")

INFO: 862 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 12 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 16 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 7 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 1 good sample after rejection sampling [thejoker.sampler.sampler]
INFO: 8 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 25 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 6 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 51 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 65 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 1 good sample after rejection sampling [thejoker.sampler.sampler]
INFO: 133 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 24 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 693 good samples after rejection samp

INFO: 562 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 726 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 548 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 180 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 2 good samples after rejection sampling [thejoker.sampler.sampler]
INFO: 2618 good samples after rejection sampling [thejoker.sampler.sampler]


In [1]:
2**16

65536

In [18]:
len(AllAvgSimp['APOGEE_ID'])

1376

In [18]:
Samples_Table = Table()
Samples_Table['P'] = samples['P']
Samples_Table['M0'] = samples['M0']
Samples_Table['e'] = samples['e']
Samples_Table['omega'] = samples['omega']
Samples_Table['jitter'] = samples['jitter']
Samples_Table['K'] = samples['K']
Samples_Table['v0'] = samples['v0']

In [19]:
Samples_Table

P,M0,e,omega,jitter,K,v0
d,rad,,rad,km / s,km / s,km / s
float64,float64,float64,float64,float64,float64,float64
22.0016033838,4.08767574816,0.652312293452,2.75568333424,0.0,1.1698884688,0.599911084935
15.2724063961,3.23563567581,0.00245158594644,3.0494458393,0.0,0.204517479497,0.715811313436
64.2117105532,1.18031884279,0.0468011453737,0.310070454576,0.0,0.401683324356,0.516584570491
15.6008182988,4.53328983349,0.0740996034178,2.51806503333,0.0,0.0781787680916,0.656168054847
69.996703528,3.90599777315,0.0134465438035,4.11122187149,0.0,0.12302476916,0.583487813571
8.39193543728,0.319871044038,0.474095720188,2.14787439893,0.0,1.04574992736,0.395361667756
43.9668629168,5.03562867319,0.834226493602,2.76834577615,0.0,0.199426660146,0.643482395954
16.9250304479,0.43595987188,0.491877769249,2.38411681748,0.0,0.388557750194,0.664824785687


In [25]:
Holder = Table(fits.getdata('/home/cody/Binary Search/2M00061700-7710022/2M00061700-7710022.fits',1))

In [7]:
data

In [2]:
compute_likelihoods(samples)

NameError: name 'samples' is not defined